questa parte serve per fare il codice di ensemble.

In [15]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

Defaulting to user installation because normal site-packages is not writeable
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
nlp = spacy.load("en_core_web_sm")

In [37]:
all_labels = ['B-STREET', 'B-CITY', 'I-DATE', 'B-PASS', 'I-CITY', 'B-TIME', 'B-EMAIL', 'I-DRIVERLICENSE',
              'I-POSTCODE', 'I-BOD', 'B-USERNAME', 'B-BOD', 'B-COUNTRY', 'B-SECADDRESS', 'B-IDCARD',
              'I-SOCIALNUMBER', 'I-PASSPORT', 'B-IP', 'O', 'B-TEL', 'B-SOCIALNUMBER', 'I-TIME', 'B-BUILDING',
              'B-PASSPORT', 'I-TITLE', 'I-SEX', 'I-STREET', 'B-STATE', 'I-STATE', 'B-TITLE', 'B-DATE',
              'B-GEOCOORD', 'I-IDCARD', 'I-TEL', 'B-POSTCODE', 'B-DRIVERLICENSE', 'I-GEOCOORD', 'I-COUNTRY',
              'I-EMAIL', 'I-PASS', 'B-SEX', 'I-USERNAME', 'I-BUILDING', 'I-IP', 'I-SECADDRESS', 'B-CARDISSUER',
              'I-CARDISSUER']

id2label = {i: l for i, l in enumerate(all_labels)}
label2id = {v: k for k, v in id2label.items()}
target = [l for l in all_labels if l != "O"]

In [23]:
from spacy.training import offsets_to_biluo_tags

In [24]:
def biluo_to_bio(tags):
    new_tags = []
    for i in tags:
        if i.startswith("U-"):
            new_tags.append(i.replace("U-", "B-"))
        elif i.startswith("L-"):
            new_tags.append(i.replace("L-", "I-"))
        else:
            new_tags.append(i)
    return new_tags

def create_bio_labels(text, spans):
    doc = nlp(text)
    spacy_tokens = [token.text for token in doc]
    spacy_tags = offsets_to_biluo_tags(doc, spans)
    if '-' in spacy_tags:
        raise Exception("Invalid tag found")
    bio_tags = biluo_to_bio(spacy_tags)
    tokenized_input = deberta_tokenizer(spacy_tokens, is_split_into_words=True, truncation=True)
    tokens = deberta_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    word_ids = tokenized_input.word_ids()
    tokenized_bio = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            tokenized_bio.append(-100)
        elif word_idx != previous_word_idx:  # Only label the first token of a given word
            tokenized_bio.append(label2id[bio_tags[word_idx]])
        else:
            tokenized_bio.append(-100)
        previous_word_idx = word_idx
    return tokens, tokenized_input["input_ids"], tokenized_bio

In [1]:
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")

/Users/lucaraffo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/lucaraffo/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})

In [26]:
document_ids = dataset['validation'][:7946]['id']
full_texts = dataset['validation'][:7946]['source_text']
span_labels = dataset['validation'][:7946]['span_labels']
from ast import literal_eval

test_dataset = [{'id':document_ids[i][:-1], 'full_text': full_texts[i], 'span_label': literal_eval(span_labels[i])} for i in range(7946)]

In [27]:
from transformers import AutoTokenizer

deberta_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge")
mbert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")

/Users/lucaraffo/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [82]:
from aux import json_to_Dataset
import json

In [86]:
with open('test.json', 'r') as file:
    data = json.load(file)


In [112]:
prova = data[8]

In [114]:
prova

{'id': '40788',
 'tokens': ['[CLS]',
  '▁Subject',
  '▁:',
  '▁Password',
  '▁Reset',
  '▁Instructions',
  '▁for',
  '▁Your',
  '▁Edu',
  'Portal',
  '▁Account',
  '▁Dear',
  '▁esteemed',
  '▁user',
  '▁,',
  '▁We',
  '▁are',
  '▁writing',
  '▁to',
  '▁inform',
  '▁you',
  '▁that',
  '▁a',
  '▁password',
  '▁reset',
  '▁has',
  '▁been',
  '▁requested',
  '▁for',
  '▁your',
  '▁Edu',
  'Portal',
  '▁account',
  '▁.',
  '▁To',
  '▁ensure',
  '▁the',
  '▁security',
  '▁of',
  '▁your',
  '▁account',
  '▁,',
  '▁please',
  '▁follow',
  '▁the',
  '▁instructions',
  '▁below',
  '▁:',
  '▁1',
  '▁.',
  '▁G',
  '46',
  '▁-',
  '▁Driver',
  "▁'",
  's',
  '▁License',
  '▁:',
  '▁THE',
  'OD',
  '910',
  '129',
  '954',
  '2',
  '▁-',
  '▁Passport',
  '▁Number',
  '▁:',
  '▁785',
  '511',
  '155',
  '▁2',
  '▁.',
  '▁m',
  'arium',
  '.',
  'demon',
  't',
  'richard',
  '195',
  '1',
  '▁-',
  '▁Driver',
  "▁'",
  's',
  '▁License',
  '▁:',
  '▁MARI',
  'U',
  '-',
  '554',
  '181',
  '▁-',
  '▁